In [28]:
import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [ ]:
dataset =pd.read_csv('./data/CICEVSE2024.csv')
data = dataset.copy()
data

In [26]:
print("Gözlem sayısı : " ,len(data))
print("Değişken sayısı : ", len(data.columns))

Gözlem sayısı :  2744700
Değişken sayısı :  72


In [4]:
data['label_mul'].value_counts()

label_mul
MaliciousEV syn stealth scan       228001
charging syn stealth               222097
charging port scan                 177409
charging service detection scan    165171
MaliciousEV port scan              158013
charging syn flood                 131106
idle tcp flood                     131104
charging push ack flood            131095
idle syn flood                     131090
charging tcp flood                 131090
charging vulnerability scan        119236
MaliciousEV service detection       87089
idle syn stealth scan               80848
idle port scan                      78145
charging os fingerprinting          72735
idle push ack flood                 65559
charging synonymous ip              65548
charging synonymous ip flood        65546
idle synonymous ip flood            65546
idle synonymous ip                  65545
MaliciousEV vulnerability scan      57058
idle vulnerability scan             55073
idle os fingerprinting              49279
idle aggressive scan    

NaN ve Inf Değerleri Kontrol Etme


In [5]:
#NaN değere sahip sutünları kaldırma
data.drop(["requested_server_name","client_fingerprint","server_fingerprint","user_agent","content_type"],axis=1,inplace=True)

In [6]:
data.drop(columns=['id', 'expiration_id', 'src_mac', 'src_oui', 'dst_mac', 'dst_oui', 'src_port','dst_port'], inplace=True)

In [7]:

non_integer_columns = data.select_dtypes(exclude=['int']).columns
print("Non-integer columns:", non_integer_columns)

Non-integer columns: Index(['src_ip', 'dst_ip', 'bidirectional_mean_ps', 'bidirectional_stddev_ps',
       'src2dst_mean_ps', 'src2dst_stddev_ps', 'dst2src_mean_ps',
       'dst2src_stddev_ps', 'bidirectional_mean_piat_ms',
       'bidirectional_stddev_piat_ms', 'src2dst_mean_piat_ms',
       'src2dst_stddev_piat_ms', 'dst2src_mean_piat_ms',
       'dst2src_stddev_piat_ms', 'application_name',
       'application_category_name', 'label_mul'],
      dtype='object')


In [8]:
#data['application_name'] = data['application_name'].round().astype(int)
#data['application_category_name'] = data['application_category_name'].round().astype(int)

data['bidirectional_mean_ps'] = data['bidirectional_mean_ps'].round().astype(int)
data['bidirectional_stddev_ps'] = data['bidirectional_stddev_ps'].round().astype(int)
data['src2dst_mean_ps'] = data['src2dst_mean_ps'].round().astype(int)
data['src2dst_stddev_ps'] = data['src2dst_stddev_ps'].round().astype(int)
data['dst2src_mean_ps'] = data['dst2src_mean_ps'].round().astype(int)
data['dst2src_stddev_ps'] = data['dst2src_stddev_ps'].round().astype(int)
data['bidirectional_mean_piat_ms'] = data['bidirectional_mean_piat_ms'].round().astype(int)
data['bidirectional_stddev_piat_ms'] = data['bidirectional_stddev_piat_ms'].round().astype(int)
data['src2dst_mean_piat_ms'] = data['src2dst_mean_piat_ms'].round().astype(int)
data['src2dst_stddev_piat_ms'] = data['src2dst_stddev_piat_ms'].round().astype(int)
data['dst2src_mean_piat_ms'] = data['dst2src_mean_piat_ms'].round().astype(int)
data['dst2src_stddev_piat_ms'] = data['dst2src_stddev_piat_ms'].round().astype(int)

In [9]:

non_integer_columns = data.select_dtypes(exclude=['int']).columns
print("Non-integer columns:", non_integer_columns)

Non-integer columns: Index(['src_ip', 'dst_ip', 'application_name', 'application_category_name',
       'label_mul'],
      dtype='object')


In [10]:
data['vlan_id'].unique(), data['dst2src_cwr_packets'].unique(), data['dst2src_urg_packets'].unique()

(array([0], dtype=int64), array([0], dtype=int64), array([0], dtype=int64))

In [11]:
from sklearn.preprocessing import LabelEncoder

# List of object columns to encode
object_columns = [
    "application_name",
    "application_category_name",
]

# Apply label encoding to each column
label_encoders = {}  # To store the encoders for each column
for col in object_columns:
    label_encoders[col] = LabelEncoder()  # Create a LabelEncoder for each column
    data[col] = label_encoders[col].fit_transform(data[col].astype(str))  # Encode the column

# Verify the result
print(data[object_columns].head())


   application_name  application_category_name
0                61                         14
1                23                          5
2                35                         13
3                22                          5
4                42                         11


In [12]:
# Original to new label mapping
label_mapping = {
    'charging Aggressive scan': 'aggressive-scan',
    'Charging Benign': 'None',
    'charging icmp flood': 'icmp-flood',
    'charging icmp fragmentation': 'icmp-fragmentation',
    'charging os fingerprinting': 'os-fingerpriting',
    'charging portscan': 'TCP-port-scan',
    'charging push ack flood': 'pshack-flood',
    'charging service detection': 'service-version-detection',
    'charging slowLoris scan': 'slowloris-scan',
    'charging syn flood': 'syn-flood',
    'charging syn stealth': 'syn-stealth-scan',
    'charging synonymous ip': 'synonymousIP-flood',
    'charging tcp flood': 'TCP-flood',
    'charging udp flood': 'upd-flood',
    'charging vulnerability scan': 'vulnerability-scan',
    'idle aggressive scan': 'aggressive-scan',
    'idle benign': 'None',
    'idle icmp fragmentation': 'icmp-fragmentation',
    'idle os fingerprinting': 'os-fingerpriting',
    'idle portscan': 'TCP-port-scan',
    'idle service detection': 'service-version-detection',
    'idle slowloris scan': 'slowloris-scan',
    'idle syn flood': 'syn-flood',
    'idle syn stealth scan': 'syn-stealth-scan',
    'idle synonymous ip': 'synonymousIP-flood',
    'idle tcp flood': 'TCP-flood',
    'idle udp flood': 'upd-flood',
    'idle vulnerability scan': 'vulnerability-scan',
    'charging aggressive scan': 'aggressive-scan',
    'charging port scan': 'TCP-port-scan',
    'charging service detection scan': 'service-version-detection',
    'charging synonymous ip flood': 'synonymousIP-flood',
    'idle icmp flood': 'icmp-flood',
    'idle port scan': 'TCP-port-scan',
    'idle push ack flood': 'pshack-flood',
    'idle synonymous ip flood': 'synonymousIP-flood',
    'MaliciousEV aggressive scan': 'aggressive-scan',
    'MaliciousEV os fingerprinting': 'os-fingerpriting',
    'MaliciousEV port scan': 'TCP-port-scan',
    'MaliciousEV service detection': 'service-version-detection',
    'MaliciousEV syn stealth scan': 'syn-stealth-scan',
    'MaliciousEV vulnerability scan': 'vulnerability-scan'
}

# Apply the mapping to the label_mul column
data['label_mul'] = data['label_mul'].map(label_mapping)

# Verify the mapping
print(data['label_mul'].unique())


['aggressive-scan' 'None' 'icmp-flood' 'icmp-fragmentation'
 'os-fingerpriting' 'TCP-port-scan' 'pshack-flood'
 'service-version-detection' 'slowloris-scan' 'syn-flood'
 'syn-stealth-scan' 'synonymousIP-flood' 'TCP-flood' 'upd-flood'
 'vulnerability-scan']


In [13]:
data['label_mul'].value_counts()

label_mul
syn-stealth-scan             530946
TCP-port-scan                426659
service-version-detection    292856
syn-flood                    262196
TCP-flood                    262194
synonymousIP-flood           262185
vulnerability-scan           231367
pshack-flood                 196654
os-fingerpriting             140273
aggressive-scan              102522
upd-flood                     32475
slowloris-scan                 4201
None                             82
icmp-flood                       62
icmp-fragmentation               28
Name: count, dtype: int64

In [ ]:
attack_types = data['label_mul'].unique()
print(attack_types)

['aggressive-scan' 'None' 'icmp-flood' 'icmp-fragmentation'
 'os-fingerpriting' 'TCP-port-scan' 'pshack-flood'
 'service-version-detection' 'slowloris-scan' 'syn-flood'
 'syn-stealth-scan' 'synonymousIP-flood' 'TCP-flood' 'upd-flood'
 'vulnerability-scan']


In [15]:
data['label_mul'].replace({ 'None':0, 'aggressive-scan':1,  
                         'icmp-flood':2, 'icmp-fragmentation':3,
                     'os-fingerpriting':4, 'TCP-port-scan':5, 'pshack-flood':6,
 'service-version-detection':7, 'slowloris-scan':8, 'syn-flood':9,
 'syn-stealth-scan':10, 'synonymousIP-flood':11, 'TCP-flood':12, 'upd-flood':13,
 'vulnerability-scan':14}, inplace= True)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2744700 entries, 0 to 2744699
Data columns (total 74 columns):
 #   Column                        Dtype 
---  ------                        ----- 
 0   src_ip                        object
 1   dst_ip                        object
 2   protocol                      int64 
 3   ip_version                    int64 
 4   vlan_id                       int64 
 5   tunnel_id                     int64 
 6   bidirectional_first_seen_ms   int64 
 7   bidirectional_last_seen_ms    int64 
 8   bidirectional_duration_ms     int64 
 9   bidirectional_packets         int64 
 10  bidirectional_bytes           int64 
 11  src2dst_first_seen_ms         int64 
 12  src2dst_last_seen_ms          int64 
 13  src2dst_duration_ms           int64 
 14  src2dst_packets               int64 
 15  src2dst_bytes                 int64 
 16  dst2src_first_seen_ms         int64 
 17  dst2src_last_seen_ms          int64 
 18  dst2src_duration_ms           int64 
 19  

In [19]:
data

,src_ip,dst_ip,protocol,ip_version,vlan_id,tunnel_id,bidirectional_first_seen_ms,bidirectional_last_seen_ms,bidirectional_duration_ms,bidirectional_packets,...,dst2src_urg_packets,dst2src_ack_packets,dst2src_psh_packets,dst2src_rst_packets,dst2src_fin_packets,application_name,application_category_name,application_is_guessed,application_confidence,label_mul
0,192.168.137.85,192.168.137.226,6,4,0,0,1703261991666,1703261991779,113,2,...,0,1,0,1,0,61,14,0,0,1
1,192.168.137.85,192.168.137.226,6,4,0,0,1703261991667,1703261991784,117,2,...,0,1,0,1,0,23,5,1,1,1
2,192.168.137.85,192.168.137.226,6,4,0,0,1703261991667,1703261991784,117,2,...,0,1,0,1,0,35,13,1,1,1
3,192.168.137.85,192.168.137.226,6,4,0,0,1703261991667,1703261991785,118,2,...,0,1,0,1,0,22,5,1,1,1
4,192.168.137.85,192.168.137.226,6,4,0,0,1703261991667,1703261991785,118,2,...,0,1,0,1,0,42,11,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2744695,192.168.10.4,192.168.10.2,6,4,0,0,1703104383605,1703104383606,1,2,...,0,1,0,1,0,61,14,0,0,14
2744696,192.168.10.4,192.168.10.2,6,4,0,0,1703104383606,1703104383606,0,2,...,0,1,0,1,0,61,14,0,0,14
2744697,192.168.10.4,192.168.10.2,6,4,0,0,1703104383606,1703104383606,0,2,...,0,1,0,1,0,61,14,0,0,14
2744698,192.168.10.4,192.168.10.2,6,4,0,0,1703104383606,1703104383606,0,2,...,0,1,0,1,0,61,14,0,0,14


In [20]:
data.to_csv("./data/cleaned_merged_data.csv", index = False)

In [23]:
data.drop(columns=['src_ip','dst_ip'], inplace=True)

In [24]:
from sklearn.preprocessing import MinMaxScaler

# Min-Max normalizasyonu için bir ölçekleyici oluşturun
scaler = MinMaxScaler()

# 'atack_num' sütununu dışındaki veri sütunlarını ölçeklendirin
scaled_data = data.copy()  # Veriyi korumak için kopyalayın
columns_to_scale = scaled_data.columns.difference(['label_mul'])  # 'atack_num' sütunu dışındaki sütunları seçin
scaled_data[columns_to_scale] = scaler.fit_transform(scaled_data[columns_to_scale])  # Seçilen sütunları ölçeklendirin

# Normalleştirilmiş veriyi görüntüleyin
print(scaled_data.head())
data = scaled_data.copy()


   protocol  ip_version  vlan_id  tunnel_id  bidirectional_first_seen_ms  \
0  0.020408         0.0      0.0        0.0                     0.187793   
1  0.020408         0.0      0.0        0.0                     0.187793   
2  0.020408         0.0      0.0        0.0                     0.187793   
3  0.020408         0.0      0.0        0.0                     0.187793   
4  0.020408         0.0      0.0        0.0                     0.187793   

   bidirectional_last_seen_ms  bidirectional_duration_ms  \
0                    0.187653                   0.000063   
1                    0.187653                   0.000065   
2                    0.187653                   0.000065   
3                    0.187653                   0.000066   
4                    0.187653                   0.000066   

   bidirectional_packets  bidirectional_bytes  src2dst_first_seen_ms  ...  \
0           9.315038e-08         9.804380e-09               0.187793  ...   
1           9.315038e-08    

In [25]:
data.to_csv("./data/cleaned_merged_data_MinMax.csv", index = False)